In [15]:
# import yfinance as yf
import feedparser
from dotenv import load_dotenv
import pandas as pd
import multiextractor
from bs4 import BeautifulSoup
import requests
import tldextract
from urllib.parse import urlparse
from datetime import datetime, date, time

load_dotenv()

True

In [4]:
r = requests.get('https://www.cnbc.com/2023/09/27/amazon-lawsuit-protects-free-and-fair-competition-ftcs-lina-khan.html')
soup = BeautifulSoup(r.content)
# b_list = soup.find('div', {'class': 'ArticleBody-articleBody'}).find('div', {'class': 'group'}).find_all('p')
# b_para = ' '.join([i.text for i in b_list])

b_art_body = multiextractor.locate_elements(soup, 'div', 'find', attrs={'class': 'ArticleBody-articleBody'})
b_list = multiextractor.process_text(b_art_body, 'summary', 'find', 'div', attrs={'class': 'group'})
b_para = ' '.join(b_list)

In [47]:
def extract_rss_body(url, body_attrs=None, list_attrs=None) -> str:
    _body_kwargs, _list_kwargs = {}, {}
    r = requests.get(url)
    soup = BeautifulSoup(r.content)
    
    if body_attrs is not None: _body_kwargs.update({'attrs': body_attrs})
    _art_body = multiextractor.locate_elements(soup, 'div', 'find', **_body_kwargs)
    
    if list_attrs is not None: _list_kwargs.update({'attrs': list_attrs})
    _body_list = multiextractor.process_text(_art_body, 'summary', 'find', 'div', **_list_kwargs)
    return ' '.join(_body_list)

def create_entry_from_rss(url, body_attrs=None, list_attrs=None) -> pd.DataFrame:
    _feeds = feedparser.parse(url)
    _entry_list = []
    for _entry in _feeds.entries:
        _entry.body = extract_rss_body(_entry.link, body_attrs=body_attrs, list_attrs=list_attrs) if (body_attrs is not None) & (list_attrs is not None) else _entry.summary
        _tmp = {
            'title': _entry.title,
            'description': _entry.summary,
            'content': _entry.body,
            'url': _entry.link,
            'image': '',
            'publishedAt': _entry.published,
            'name': tldextract.extract(_entry.link).domain.title(),
            'domainName': urlparse(_entry.link).netloc,
            'publishedDate': date(_entry.published_parsed.tm_year, _entry.published_parsed.tm_mon, _entry.published_parsed.tm_mday),
            'publishedTime': time(_entry.published_parsed.tm_hour, _entry.published_parsed.tm_min, _entry.published_parsed.tm_sec)
        }
        _entry_list.append(_tmp)
    _df = pd.DataFrame(_entry_list)
    _df['publishedAt'] = pd.to_datetime(_df['publishedAt']).dt.strftime('%Y%m%d %H:%M:%S%z+00:00')
    return _df

In [32]:
# cnbc
url = 'https://search.cnbc.com/rs/search/combinedcms/view.xml?partnerId=wrss01&id=19854910'
cnbc_arts = create_entry_from_rss(url, body_attrs={'class': 'ArticleBody-articleBody'}, list_attrs={'class': 'group'})

In [48]:
# trading economics
url = 'https://tradingeconomics.com/rss/news.aspx?i=bank+lending+rate'
trade_econ_blr = create_entry_from_rss(url)

In [49]:
trade_econ_blr

,title,description,content,url,image,publishedAt,name,domainName,publishedDate,publishedTime
0,Thailand Prime Lending Rate,Bank Lending Rate in Thailand remained unchang...,Bank Lending Rate in Thailand remained unchang...,https://tradingeconomics.com/thailand/bank-len...,,20231002 01:09:07+0000+00:00,Tradingeconomics,tradingeconomics.com,2023-10-02,01:09:07
1,Botswana Bank Lending Rate,Bank Lending Rate in Botswana remained unchang...,Bank Lending Rate in Botswana remained unchang...,https://tradingeconomics.com/botswana/bank-len...,,20231002 01:07:55+0000+00:00,Tradingeconomics,tradingeconomics.com,2023-10-02,01:07:55
2,Malaysia Bank Lending Rate,Bank Lending Rate in Malaysia decreased to 5.4...,Bank Lending Rate in Malaysia decreased to 5.4...,https://tradingeconomics.com/malaysia/bank-len...,,20230930 10:04:04+0000+00:00,Tradingeconomics,tradingeconomics.com,2023-09-30,10:04:04
3,Israel Bank Prime Rate,Bank Lending Rate in Israel increased to 5.88 ...,Bank Lending Rate in Israel increased to 5.88 ...,https://tradingeconomics.com/israel/bank-lendi...,,20230930 10:04:01+0000+00:00,Tradingeconomics,tradingeconomics.com,2023-09-30,10:04:01
4,Finland Bank Lending Rate,Bank Lending Rate in Finland decreased to 7.74...,Bank Lending Rate in Finland decreased to 7.74...,https://tradingeconomics.com/finland/bank-lend...,,20230930 10:03:59+0000+00:00,Tradingeconomics,tradingeconomics.com,2023-09-30,10:03:59
5,China Bank Lending Rate 6 Months to 1 Year,China Bank Lending Rate 6 Months to 1 Year was...,China Bank Lending Rate 6 Months to 1 Year was...,https://tradingeconomics.com/china/bank-lendin...,,20230929 06:05:28+0000+00:00,Tradingeconomics,tradingeconomics.com,2023-09-29,06:05:28
6,Egypt Bank Lending Rate,Bank Lending Rate in Egypt increased to 18.70 ...,Bank Lending Rate in Egypt increased to 18.70 ...,https://tradingeconomics.com/egypt/bank-lendin...,,20230928 16:05:14+0000+00:00,Tradingeconomics,tradingeconomics.com,2023-09-28,16:05:14
7,Ukraine Bank Lending Rate,Bank Lending Rate in Ukraine increased to 21.0...,Bank Lending Rate in Ukraine increased to 21.0...,https://tradingeconomics.com/ukraine/bank-lend...,,20230927 16:05:20+0000+00:00,Tradingeconomics,tradingeconomics.com,2023-09-27,16:05:20
8,Sri Lanka Prime Lending Rate,Bank Lending Rate in Sri Lanka decreased to 15...,Bank Lending Rate in Sri Lanka decreased to 15...,https://tradingeconomics.com/sri-lanka/bank-le...,,20230927 16:05:19+0000+00:00,Tradingeconomics,tradingeconomics.com,2023-09-27,16:05:19
9,Brazil Bank Lending Rate,Bank Lending Rate in Brazil decreased to 57.71...,Bank Lending Rate in Brazil decreased to 57.71...,https://tradingeconomics.com/brazil/bank-lendi...,,20230927 16:05:13+0000+00:00,Tradingeconomics,tradingeconomics.com,2023-09-27,16:05:13


In [42]:
# cnbc
url = 'https://search.cnbc.com/rs/search/combinedcms/view.xml?partnerId=wrss01&id=19854910'
feed = feedparser.parse(url)
for entry in feed.entries:
    entry.title
    entry.title_detail.language
    entry.link
    entry.summary
    entry.summary_detail.language
    entry.published
    pub_year = entry.published_parsed.tm_year
    pub_mth = entry.published_parsed.tm_mon
    pub_day = entry.published_parsed.tm_mday
    pub_hr = entry.published_parsed.tm_hour
    pub_min = entry.published_parsed.tm_min
    pub_sec = entry.published_parsed.tm_sec
    pub_wkday = entry.published_parsed.tm_wday

In [43]:
entry

{'links': [{'rel': 'alternate',
   'type': 'text/html',
   'href': 'https://www.cnbc.com/2023/09/26/barry-diller-says-fair-use-needs-to-be-redefined-to-address-ai.html'}],
 'link': 'https://www.cnbc.com/2023/09/26/barry-diller-says-fair-use-needs-to-be-redefined-to-address-ai.html',
 'id': '107306565',
 'guidislink': False,
 'metadata_type': 'cnbcnewsstory',
 'metadata_id': '107306565',
 'metadata_sponsored': 'false',
 'title': 'Barry Diller rips WGA deal with studios, says fair use needs to be redefined to address AI',
 'title_detail': {'type': 'text/plain',
  'language': None,
  'base': 'https://search.cnbc.com/rs/search/combinedcms/view.xml?partnerId=wrss01&id=19854910',
  'value': 'Barry Diller rips WGA deal with studios, says fair use needs to be redefined to address AI'},
 'summary': 'Copyright law and fair use must be redefined to exclude AI and protect publishers from their content being "sucked up," Barry Diller said Tuesday on Squawk Box.',
 'summary_detail': {'type': 'text/h

In [39]:
# tradingeconomics
url = 'https://tradingeconomics.com/rss/news.aspx?i=bank+lending+rate'
feed = feedparser.parse(url)
for entry in feed.entries:
    entry.title
    entry.title_detail.language
    entry.link
    entry.summary
    entry.published
    pub_year = entry.published_parsed.tm_year
    pub_mth = entry.published_parsed.tm_mon
    pub_day = entry.published_parsed.tm_mday
    pub_hr = entry.published_parsed.tm_hour
    pub_min = entry.published_parsed.tm_min
    pub_sec = entry.published_parsed.tm_sec
    pub_wkday = entry.published_parsed.tm_wday

In [41]:
entry

{'title': 'Singapore Prime Lending Rate',
 'title_detail': {'type': 'text/plain',
  'language': None,
  'base': 'https://tradingeconomics.com/rss/news.aspx?i=bank+lending+rate',
  'value': 'Singapore Prime Lending Rate'},
 'links': [{'rel': 'alternate',
   'type': 'text/html',
   'href': 'https://tradingeconomics.com/singapore/bank-lending-rate'}],
 'link': 'https://tradingeconomics.com/singapore/bank-lending-rate',
 'summary': 'Bank Lending Rate in Singapore remained unchanged at 5.25 percent in June from 5.25 percent in May of 2021. Bank Lending Rate in Singapore averaged 6.51 percent from 1978 until 2021, reaching an all time high of 14.98 percent in October of 1981 and a record low of 5.25 percent in March of 2019. In Singapore, the prime lending rate is the average rate of interest charged on loans by 10 Singaporean leading banks and finance companies to private individuals and companies. This page provides - Singapore Prime Lending Rate - actual values, historical data, forecast,

In [25]:
import os
import psycopg2

USER = os.getenv('COCKROACH_USER')
PW = os.getenv('COCKROACH_PW')
HOST = os.getenv('COCKROACH_HOST')
PORT = os.getenv('COCKROACH_PORT')
DB = os.getenv('COCKROACH_DB')
conn_str = f'postgresql://{USER}:{PW}@{HOST}:{PORT}/{DB}'

# conn = psycopg2.connect(conn_str)
# with conn.cursor() as cur:
#     cur.execute("SELECT now()")
#     res = cur.fetchall()
#     conn.commit()
#     print(res)

[(datetime.datetime(2023, 9, 30, 8, 43, 14, 739815, tzinfo=datetime.timezone.utc),)]
